In [20]:
import numpy as np
import pandas as pd
import psycopg2
from scipy.optimize import fsolve

### Constants

In [21]:
g = 9.81
rho = 1000
k = 0.3 # erodibility constant
b = 1.5 # empirical constant

### Function for wavelength equation

In [22]:
def wavelength_eqn(lambda_, T, d):
    k = 2 * np.pi / lambda_  # Wave number
    return (2 * np.pi / T) ** 2 - g * k * np.tanh(k * d)

In [23]:
def compute_wavelength(T, d):
    deep_water_lambda = (g * T**2) / (2 * np.pi)  # deep water approximation
    shallow_water_lambda = T * np.sqrt(g * d)  # shallow water approximation

    if d > deep_water_lambda / 2:
        return deep_water_lambda  # deep water case
    elif d < shallow_water_lambda / 20:
        return shallow_water_lambda  # shallow water case
    else:
        return fsolve(wavelength_eqn, deep_water_lambda, args=(T, d))[0] 

### Generate synthetic dataset

In [24]:
num_samples = 10000
time_periods = np.random.uniform(1, 7, num_samples)  
depths = np.random.uniform(0.5, 5, num_samples)  
# wave height = 0.78 * depth
wave_heights = np.random.uniform(0.5 * 0.78, 5 * 0.78, num_samples)

In [25]:
wavelengths = []
velocities = []
energies = []
powers = []
shear_stresses = []
impact_pressures = []
soil_erosion_rates = []

In [26]:
for T, d, H in zip(time_periods, depths, wave_heights):
    lambda_value = compute_wavelength(T, d)  # Compute wavelength
    velocity = lambda_value / T  # Wave velocity
    energy = 0.5 * rho * g * H**2  # Wave energy
    power = energy * velocity  # Wave power
    shear_stress = (rho * g * H) / d  # Shear stress
    soil_erosion_rate = k * (shear_stress ** b)  # Soil erosion rate
    impact_pressure = rho * (velocity ** 2)  # Impact pressure

    wavelengths.append(lambda_value)
    velocities.append(velocity)
    energies.append(energy)
    powers.append(power)
    shear_stresses.append(shear_stress)
    soil_erosion_rates.append(soil_erosion_rate)
    impact_pressures.append(impact_pressure)

/tmp/ipykernel_12286/2941639387.py:10: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  return fsolve(wavelength_eqn, deep_water_lambda, args=(T, d))[0]


In [27]:
df = pd.DataFrame({
    'time_period': time_periods,
    'depth': depths,
    'wave_height': wave_heights,
    'wavelength': wavelengths,
    'velocity': velocities,
    'energy': energies,
    'power': powers,
    'shear_stress': shear_stresses,
    'soil_erosion_rate': soil_erosion_rates,
    'impact_pressure': impact_pressures
})

In [28]:
df.head()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
0,2.736741,0.810422,1.382149,7.154082,2.614088,9370.192710,24494.510891,16730.636972,649217.127011,6833.457506
1,2.375450,0.601222,1.262668,5.354894,2.254265,7820.195133,17628.792083,20602.683300,887170.178169,5081.710634
2,4.390990,2.419754,3.345947,19.584686,4.460198,54913.231755,244923.904767,13564.908686,473965.321188,19893.369175
3,1.622785,1.586119,2.467435,4.051960,2.496917,29862.789382,74564.902307,15260.852458,565574.011511,6234.593808
4,1.504657,2.362584,2.570541,3.534794,2.349236,32410.682774,76140.338101,10673.489111,330811.708127,5518.909109


In [29]:
df.describe()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000
mean,4.014871,2.751289,2.140214,14.111751,3.402532,27530.848219,94798.998106,10715.837968,4.238689e+05,17476.709731
std,1.737221,1.306672,1.016072,14.840507,2.429008,21818.803683,112508.014007,9954.870461,6.508397e+05,10132.240588
min,1.000526,0.500041,0.390070,-38.398574,-5.490898,746.317583,-400934.754338,833.221604,7.215427e+03,2387.622643
25%,2.522753,1.621925,1.262293,6.111209,2.676974,7815.542404,19922.741191,4563.630400,9.248833e+04,8767.957484
50%,4.025738,2.763421,2.133241,14.449520,3.796682,22321.263180,71522.128839,7580.019867,1.979825e+05,15988.041639
75%,5.507369,3.882630,3.015607,23.631039,4.969841,44605.521212,159641.473053,12980.417153,4.436640e+05,25115.876415
max,6.998536,4.999090,3.899614,60.010626,9.673151,74590.267022,466039.155927,73919.187161,6.029163e+06,93569.849655


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time_period        10000 non-null  float64
 1   depth              10000 non-null  float64
 2   wave_height        10000 non-null  float64
 3   wavelength         10000 non-null  float64
 4   velocity           10000 non-null  float64
 5   energy             10000 non-null  float64
 6   power              10000 non-null  float64
 7   shear_stress       10000 non-null  float64
 8   soil_erosion_rate  10000 non-null  float64
 9   impact_pressure    10000 non-null  float64
dtypes: float64(10)
memory usage: 781.4 KB


In [31]:
df = df[(df >= 0).all(axis=1)]

In [32]:
df.head()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
0,2.736741,0.810422,1.382149,7.154082,2.614088,9370.192710,24494.510891,16730.636972,649217.127011,6833.457506
1,2.375450,0.601222,1.262668,5.354894,2.254265,7820.195133,17628.792083,20602.683300,887170.178169,5081.710634
2,4.390990,2.419754,3.345947,19.584686,4.460198,54913.231755,244923.904767,13564.908686,473965.321188,19893.369175
3,1.622785,1.586119,2.467435,4.051960,2.496917,29862.789382,74564.902307,15260.852458,565574.011511,6234.593808
4,1.504657,2.362584,2.570541,3.534794,2.349236,32410.682774,76140.338101,10673.489111,330811.708127,5518.909109


In [33]:
df.describe()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
count,9271.000000,9271.000000,9271.000000,9271.000000,9271.000000,9271.000000,9271.000000,9271.000000,9.271000e+03,9271.000000
mean,3.901539,2.820421,2.146442,17.021122,3.986187,27663.452286,110468.564187,10509.227169,4.135112e+05,17522.447452
std,1.725643,1.316604,1.016241,10.743305,1.277865,21844.726196,98123.664748,9893.707802,6.468529e+05,10357.180317
min,1.000526,0.500041,0.390070,1.562952,1.545193,746.317583,1287.731973,833.221604,7.215427e+03,2387.622643
25%,2.395978,1.669140,1.269801,7.890085,2.927199,7908.790739,28504.363806,4458.393709,8.930770e+04,8568.496938
50%,3.853033,2.903858,2.144937,15.686767,3.979907,22566.695084,82323.534690,7414.516783,1.915338e+05,15839.656950
75%,5.357612,3.964654,3.020957,24.508804,5.063046,44763.927619,167572.872923,12569.085783,4.227434e+05,25634.439082
max,6.998536,4.999090,3.899614,60.010626,9.673151,74590.267022,466039.155927,73919.187161,6.029163e+06,93569.849655
